In [1]:
import pandas as pd
import numpy as np
import re
import pickle as pkl

In [2]:
# valid_name = "../output/valid_compare_16.v2.csv"
valid_name = "../output/train_compare_16.v2.csv"

In [3]:
train_df = pd.read_csv('../input/en_train.csv')

In [8]:
valid_df = pd.read_csv(valid_name)
valid_df.shape

(6415, 4)

In [5]:
train_df.iloc[[885, 886, 887]]

,sentence_id,token_id,class,before,after
885,67,0,PLAIN,In,In
886,67,1,DATE,2010,twenty ten
887,67,2,PLAIN,Sadovyi,Sadovyi


In [6]:
train_df.iloc[[33087, 33088, 33089, 33090, 33091, 33092]]

,sentence_id,token_id,class,before,after
33087,2631,11,PLAIN,Exceptions,Exceptions
33088,2631,12,PLAIN,Report,Report
33089,2631,13,CARDINAL,2010,two thousand ten
33090,2631,14,PUNCT,-,-
33091,2631,15,CARDINAL,2011,two thousand eleven
33092,2631,16,PUNCT,",",","


In [25]:
train_df.iloc[[36938, 36939, 36940, 36941, 36942]]

,sentence_id,token_id,class,before,after
36938,2941,7,PLAIN,in,in
36939,2941,8,CARDINAL,2010,two thousand ten
36940,2941,9,PUNCT,-,-
36941,2941,10,CARDINAL,2011,two thousand eleven
36942,2941,11,PLAIN,at,at


In [7]:
valid_df

,after,before,true,class_pred
0,lead two sulphate,PbSO4,PLAIN,LETTERS
1,p o m z,POMZ,LETTERS,PLAIN
2,b o e v,Boev,LETTERS,PLAIN
3,SHU,SHU,PLAIN,LETTERS
4,p i a l a t,PIALAT,LETTERS,PLAIN
5,one thousand nine hundred eighteen,1918,CARDINAL,DATE
6,corporal,CPL,PLAIN,LETTERS
7,j o e p,Joep,LETTERS,PLAIN
8,ITA,ITA,PLAIN,LETTERS
9,W,W,PLAIN,LETTERS


In [20]:
labels = ['PLAIN', 'PUNCT', 'DATE', 'LETTERS', 'CARDINAL', 'VERBATIM',
          'DECIMAL', 'MEASURE', 'MONEY', 'ORDINAL', 'TIME', 'ELECTRONIC',
          'DIGIT', 'FRACTION', 'TELEPHONE', 'ADDRESS']
label2index = dict(zip(labels, range(len(labels))))
index2label = dict([(v, k) for k, v in label2index.items()])

In [13]:
def is_com(token):
    token = str(token)
    pt = re.compile(r"\.[a-zA-Z]{2,}")
    return True if pt.search(token) else False

In [12]:
def is_year(token):
    token = str(token)
    return token.isdigit() and len(token) == 4 and int(token) <= 3000 and int(token) >= 1000

In [21]:
with open("../output/letter_set.pkl", "rb") as f:
    letter_set = pkl.load(f)

In [22]:
count = 0
for i, row in valid_df.iterrows():
    # 1.被误认为是cardinal的实际上的date的年份
    if row["class_pred"] == "CARDINAL":
        if is_year(row["before"]):
            # 不能直接修改row，因为row是副本
            valid_df.loc[i, "class_pred"] = "DATE"
            count += 1
    elif row["class_pred"] == "PLAIN":
        # 2.被误认为是plain的实际上的electronic的网站
        if is_com(row["before"]):
            valid_df.loc[i, "class_pred"] = "ELECTRONIC"
            count += 1
        # 3.被误认为是plain的实际上是letters
        elif row["before"] in letter_set:
            valid_df.loc[i, "class_pred"] = "LETTERS"
            count += 1
    elif row["class_pred"] == "LETTERS":
        if row["before"] not in letter_set:
            valid_df.loc[i, "class_pred"] = "PLAIN"
            count += 1
print(count)

4589


In [14]:
card_df = train_df.loc[np.logical_and(train_df["class"] == "CARDINAL", train_df["before"].apply(is_year))]

In [15]:
card_next_df = train_df.loc[card_df.index+1]
card_next_df.shape

(4066, 5)

In [18]:
card_next_df["before"].value_counts()

-                 1553
/                  692
:                  330
"                  263
.                  254
,                  139
season              83
)                   79
school              58
(                   38
and                 18
in                  14
people              12
player              11
he                   9
metres               9
Season               8
males                8
to                   7
females              7
was                  7
year                 6
academic             6
is                   5
with                 5
a                    5
meters               5
at                   5
b                    5
by                   5
                  ... 
-1975                1
under                1
XT                   1
-1955                1
Sheridan             1
-1967                1
households           1
Check                1
within               1
pages                1
Commonwealth         1
Thornton             1
Greeks     

In [24]:
1553. / 4066 

0.381947860304968

In [1]:
139. / 4066 

0.03418593212001968

In [16]:
date_df = train_df.loc[np.logical_and(train_df["class"] == "DATE", train_df["before"].apply(is_year))]

In [17]:
date_next_df = train_df.loc[date_df.index+1]
date_next_df.shape

(147608, 5)

In [23]:
4818 / 147608.

0.0326405072895778

In [19]:
date_next_df["before"].value_counts()

)                 34796
.                 31937
,                 28507
-                  4818
and                4674
to                 3108
"                  2974
he                 2166
:                  1625
the                1591
census             1500
by                 1027
(                  1017
in                  852
as                  630
season              623
;                   536
with                517
Summer              425
it                  421
was                 414
she                 397
Census              389
World               373
at                  360
until               351
for                 349
when                326
a                   291
film                256
                  ...  
Taniguchi             1
Brill                 1
KJDL                  1
Hrdy                  1
Niel                  1
Mexborough            1
Greater               1
operation             1
Suomi                 1
palm                  1
Manju           

In [47]:
next_df.loc[next_df["before"] == "-"]

,sentence_id,token_id,class,before,after
5452,441,20,PUNCT,-,-
6262,507,10,PUNCT,-,-
18064,1465,16,PUNCT,-,-
18793,1519,18,PUNCT,-,-
19730,1589,7,PUNCT,-,-
19868,1600,14,PUNCT,-,-
32379,2574,5,PUNCT,-,-
32961,2620,11,PUNCT,-,-
33090,2631,14,PUNCT,-,-
36940,2941,9,PUNCT,-,-


In [37]:
train_df.iloc[[9868011, 9868012, 9868013]]

,sentence_id,token_id,class,before,after
9868011,744362,7,PLAIN,the,the
9868012,744362,8,CARDINAL,2007,two thousand seven
9868013,744362,9,PUNCT,-,-


In [45]:
diff_df = valid_df.loc[valid_df["true"] != valid_df["class_pred"]]
diff_df.shape

(1847, 4)

In [10]:
diff_df["true"].value_counts()

CARDINAL     912
DIGIT        338
LETTERS      300
DECIMAL      136
VERBATIM      84
PLAIN         40
PUNCT         15
DATE          12
ORDINAL        5
TIME           1
FRACTION       1
ADDRESS        1
TELEPHONE      1
MONEY          1
Name: true, dtype: int64

In [11]:
diff_df["class_pred"].value_counts()

DATE          826
CARDINAL      452
PLAIN         374
DIGIT         119
LETTERS        25
PUNCT          25
VERBATIM       20
TELEPHONE       2
MEASURE         1
ELECTRONIC      1
ORDINAL         1
MONEY           1
Name: class_pred, dtype: int64

In [18]:
diff_df

,after,before,true,class_pred
0,o seven,07,DIGIT,CARDINAL
3,utenlandskorrespondentene,utenlandskorrespondentene,PLAIN,ELECTRONIC
5,p t x,ptx,VERBATIM,LETTERS
9,one,I,CARDINAL,PLAIN
11,one thousand nine hundred four,1904,CARDINAL,DATE
15,nine,9,DIGIT,CARDINAL
17,one thousand nine hundred seventy eight,1978,CARDINAL,DATE
18,to,-,PLAIN,PUNCT
19,two hundred eighty three,283,DECIMAL,CARDINAL
20,two,2,DIGIT,CARDINAL


In [17]:
diff_df.loc[diff_df["true"] == "DIGIT"]

,after,before,true,class_pred
0,o seven,07,DIGIT,CARDINAL
15,nine,9,DIGIT,CARDINAL
20,two,2,DIGIT,CARDINAL
24,three three o o,3300,DIGIT,CARDINAL
43,eight one two three,8123,DIGIT,CARDINAL
49,four one,41,DIGIT,CARDINAL
59,o nine o seven three one eight o three seven,0907318037,DIGIT,TELEPHONE
103,four o two,402,DIGIT,CARDINAL
109,one o o nine,1009,DIGIT,DATE
117,two o one one,2011,DIGIT,DATE


In [12]:
diff_df.loc[diff_df["true"] == "CARDINAL"]

,after,before,true,class_pred
5,one thousand nine hundred eighteen,1918,CARDINAL,DATE
11,one thousand nine hundred eight,1908,CARDINAL,DATE
12,one thousand nine hundred twenty three,1923,CARDINAL,DATE
13,one thousand nine hundred twenty four,1924,CARDINAL,DATE
15,two thousand nine,2009,CARDINAL,DATE
25,two thousand fifteen,2015,CARDINAL,DATE
34,one thousand nine hundred eighty eight,1988,CARDINAL,DATE
35,one thousand nine hundred eighty nine,1989,CARDINAL,DATE
37,two thousand eight,2008,CARDINAL,DATE
38,two thousand three,2003,CARDINAL,DATE


In [14]:
diff_df.loc[np.logical_and(diff_df["true"] == "DIGIT", diff_df["class_pred"] == "CARDINAL")]

,after,before,true,class_pred
0,o seven,07,DIGIT,CARDINAL
15,nine,9,DIGIT,CARDINAL
20,two,2,DIGIT,CARDINAL
24,three three o o,3300,DIGIT,CARDINAL
43,eight one two three,8123,DIGIT,CARDINAL
49,four one,41,DIGIT,CARDINAL
103,four o two,402,DIGIT,CARDINAL
142,one four,14,DIGIT,CARDINAL
145,two,2,DIGIT,CARDINAL
163,one nine,19,DIGIT,CARDINAL


In [15]:
diff_df.loc[diff_df["true"] == "LETTERS"]

,after,before,true,class_pred
36,n e,NE,LETTERS,PLAIN
90,I,I,LETTERS,PLAIN
150,v,v,LETTERS,PLAIN
171,a h,AH,LETTERS,PLAIN
177,D,D,LETTERS,PLAIN
185,p b,pb,LETTERS,VERBATIM
187,n g,ng,LETTERS,PLAIN


In [10]:
lp_df = diff_df.loc[np.logical_and(diff_df["true"] == "LETTERS", diff_df["class_pred"] == "PLAIN")]
lp_df.shape

(643, 3)

In [11]:
pl_df = diff_df.loc[np.logical_and(diff_df["true"] == "PLAIN", diff_df["class_pred"] == "LETTERS")]
pl_df.shape

(377, 3)

In [14]:
with open("../output/letter_set.pkl", "rb") as f:
    letter_set = pkl.load(f)
len(letter_set)

224397

In [15]:
count = 0
for one in lp_df["before"].values:
    if one in letter_set:
        count += 1
print(count)

590


In [16]:
count = 0
for one in pl_df["before"].values:
    if one in letter_set:
        count += 1
print(count)

0
